In [40]:
import requests
import pandas as pd

In [59]:
def get_historical_bitcoin_data():
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=30&interval=daily&precision=0"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error al obtener los datos:", response.status_code)
        return None


In [60]:
historical_data = get_historical_bitcoin_data()

df = pd.DataFrame(historical_data)
df

,prices,market_caps,total_volumes
0,"[1689638400000, 30148]","[1689638400000, 585500720965.1113]","[1689638400000, 10974383371.546566]"
1,"[1689724800000, 29848]","[1689724800000, 579708613537.5165]","[1689724800000, 13152758553.030825]"
2,"[1689811200000, 29920]","[1689811200000, 581327322356.8716]","[1689811200000, 8545620844.326587]"
3,"[1689897600000, 29787]","[1689897600000, 579308712612.258]","[1689897600000, 11117456853.464512]"
4,"[1689984000000, 29915]","[1689984000000, 581397880391.4172]","[1689984000000, 8492627980.9282]"
5,"[1690070400000, 29710]","[1690070400000, 577068525402.119]","[1690070400000, 6745541064.873876]"
6,"[1690156800000, 30058]","[1690156800000, 584296796267.4817]","[1690156800000, 7427606999.412798]"
7,"[1690243200000, 29185]","[1690243200000, 567051778655.836]","[1690243200000, 15195244950.838041]"
8,"[1690329600000, 29223]","[1690329600000, 568019115128.7598]","[1690329600000, 7929844775.363448]"
9,"[1690416000000, 29364]","[1690416000000, 570531781624.6747]","[1690416000000, 13088677750.42505]"
